# MySQL 8.0.40

## 0) 前提

* エンジン: **MySQL 8**
* 並び順: 任意（`ORDER BY` を付けない）
* `NOT IN` は NULL 罠のため回避
* 判定は **ID 基準**（ここでは `class`）、表示は仕様どおりの列名と順序

## 1) 問題

* `Courses` から **受講生が5人以上いる class を求める**
* 入力テーブル例: `Courses(student, class)`（主キー: `(student, class)`）
* 出力仕様: 列は `class` のみ／順序任意／重複なし

## 2) 最適解（単一クエリ）

> ウィンドウ関数で各クラスの人数を数え、閾値で抽出して最終投影。

```sql
WITH win AS (
  SELECT
    class,
    COUNT(*) OVER (PARTITION BY class) AS cnt
  FROM Courses
)
SELECT DISTINCT
  class
FROM win
WHERE cnt >= 5;

Runtime 311 ms
Beats 58.52%

```

* `(student, class)` が PK のため同一学生の同一クラス重複は存在せず、`COUNT(*)` で十分
* 結果順は任意なので `ORDER BY` なし

## 3) 代替解

> 単純集約で十分なサイズ・要件なら `GROUP BY ... HAVING` が最軽量。

```sql
SELECT
  class
FROM Courses
GROUP BY class
HAVING COUNT(*) >= 5;

Runtime 308 ms
Beats 62.45%

```

## 4) 要点解説

* **方針**: クラス単位で人数を数え、しきい値（5）以上のみ返す
* **NULL / 重複**: 主キー制約により `(student, class)` の重複はなし。`class` 自体が NULL の行がある想定なら、条件側で `class IS NOT NULL` を併記（今回は問題仕様上不要）
* **安定性**: 並び順指定なしで I/O を節約

## 5) 計算量（概算）

* ウィンドウ（最適解）: パーティション内で **O(N)**〜**O(N log N)**（実装依存）
* 集約（代替解）: ハッシュ集約で **O(N)** 近似

## 6) 図解（Mermaid 超保守版）

```mermaid
flowchart TD
  A[入力 テーブル Courses] --> B[クラス単位で人数カウント]
  B --> C[人数が5以上を抽出]
  C --> D[出力 class 列のみ]
```
結論から言うと、この問題では **`GROUP BY ... HAVING` が最もシンプルで、実務でもまずこれを使います。**
ただし速度面をもう一段伸ばす余地はあります。

## 速くするための実務的ポイント

### 1) 二次インデックスを追加（最有効）

`GROUP BY class` の集約を軽くするには **`class` にインデックス**を張るのが一番効きます。

```sql
-- 目的: クラス単位の集約をインデックス範囲走査で処理
CREATE INDEX idx_courses_class ON Courses(class);
-- 余力があれば順序付与用に
-- CREATE INDEX idx_courses_class_student ON Courses(class, student);
```

* PK が `(student, class)` なので、そのままだと `class` での集約に不利。
* `idx_courses_class` があると MySQL は **インデックス順に走査しながらグループ化**でき、
  場合によっては **テンポラリやファイルソートを回避**します（`EXPLAIN` で `Using index for group-by` を目指す）。

### 2) クエリは `GROUP BY ... HAVING` を採用

ウィンドウ関数版は **同じクラスの行を全て数えてから DISTINCT** するので一手間多く、通常やや不利です。
以下で十分最適です。

```sql
SELECT
  class
FROM Courses
GROUP BY class
HAVING COUNT(*) >= 5;
```

### 3) 「存在判定」最適化（インデックスがある前提の代替案）

**「5件目が存在するか」だけ**を確かめる相関サブクエリは、クラスごとに **最大5行だけ**見れば良いので、
**`(class)` か `(class, student)` インデックス**がある環境では速くなることがあります。

```sql
-- (class, student) インデックスがあると更に安定
SELECT DISTINCT c.class
FROM Courses c
WHERE EXISTS (
  SELECT 1
  FROM Courses i
  WHERE i.class = c.class
  ORDER BY i.student
  LIMIT 4, 1  -- 5件目が取れれば「5人以上」と判定
);
```

> 注意: これは **インデックスの効き**に強く依存します。`EXPLAIN` で内側が `range` / `ref` になっているか確認を。

### 4) 実行計画チェック

`EXPLAIN` で見るポイント

* `type`: `range` / `ref`（全表 `ALL` は避けたい）
* `key`: 上記の新インデックスが選ばれているか
* `Extra`: `Using index for group-by` が出るとご機嫌

## まとめ（提案の優先度）

1. ✅ **`CREATE INDEX idx_courses_class (class)` を追加**
2. ✅ 本番クエリは **`GROUP BY ... HAVING COUNT(*) >= 5`**
3. ⭕ 負荷やデータ分布次第で、**`EXISTS + LIMIT 4,1`** 案を A/B して速い方を採用

この3点で、提示の ~310ms からの短縮が十分見込めます。インデックス追加が難しい（LeetCode 等）なら、現状の **`GROUP BY ... HAVING` が最適解**です。
